In [9]:
import sys
import os
sys.path.append(sys.path[0] + "/..")

%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pandas as pd
import numpy as np
import networkx as nx
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM
import math
import matplotlib.pyplot as plt
from matplotlib import gridspec
import seaborn as sns
from jupyterthemes import jtplot
import datetime
jtplot.style()

from hedgepy.core import build_series, build_network_time_series
from hedgepy.utils import write_series, read_series, read_data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# read in Dow Jones stocks data
df_dja = pd.read_csv('../experiments/data/interim/DJA-2000-2020-clean.csv')
df_dja['date'] = pd.to_datetime(df_dja['date'])
df_dja.set_index('date', inplace=True)
df_dja = df_dja.drop('Unnamed: 0', axis=1)
df_dja.head()

,open,high,low,close,name
date,,,,,
2000-05-11,101.37,104.25,99.00,102.81,AAPL
2000-05-12,106.00,110.50,104.77,107.62,AAPL
2000-05-15,108.06,108.06,100.12,101.00,AAPL
2000-05-16,104.52,109.06,102.75,105.69,AAPL
2000-05-17,103.62,103.69,100.37,101.37,AAPL


In [13]:
import yfinance as yf
dja_ticker_lst = [
    'aapl', 
    'axp', 
    'ba', 
    'cat', 
    'csco', 
    'cvx', 
    'dis',
    'dow', 
    'gs', 
    'hd', 
    'ibm', 
    'intc', 
    'jnj', 
    'jpm', 
    'ko', 
    'mcd',
    'mmm', 
    'mrk', 
    'msft', 
    'nke', 
    'pfe', 
    'pg', 
    'rtx', 
    'trv', 
    'unh', 
    'v', 
    'vz', 
    'wba', 
    'wmt', 
    'xom'
]

tickers = [yf.Ticker(ticker).history() for ticker in dja_ticker_lst]

- MCDMMM: No data found, symbol may be delisted


In [15]:
t = yf.Ticker('aapl').history(start="2017-01-01", end="2017-04-30")

In [7]:
# check # of data points for each stock
stocks = df_dja['name'].unique()
for i in stocks:
    print(len(df_dja[df_dja['name'] == i]), i)

5032 AAPL
5033 AXP
5033 BA
5033 CAT
5032 CSCO
5033 CVX
5033 DIS
291 DOW
5033 GS
5033 HD
5033 IBM
5032 INTC
5033 JNJ
5033 JPM
5033 KO
5033 MCD
5033 MMM
5033 MRK
5032 MSFT
5033 NKE
5033 PFE
5033 PG
5033 RTX
5033 TRV
5033 UNH
3060 V
5033 VZ
5032 WBA
5033 WMT
5033 XOM


In [8]:
# remove Dow and Verizon
df_dja = df_dja[(df_dja['name'] != 'V') & (df_dja['name'] != 'DOW')]

In [23]:
!python -m pip install --index-url=https://bloomberg.bintray.com/pip/simple blpapi

Looking in indexes: https://bloomberg.bintray.com/pip/simple


In [25]:
import pdblp
con = pdblp.BCon(debug=True, port=8194, timeout=5000)
con.start()

ImportError: 
---------------------------- ENVIRONMENT -----------------------------
Platform: Windows-10-10.0.18362-SP0
Architecture: ('64bit', 'WindowsPE')
Python: 3.7.3 (default, Apr 24 2019, 15:29:51) [MSC v.1915 64 bit (AMD64)]
Python implementation: CPython

blpapi 64-bit will be loaded from: "C:\blp\DAPI\blpapi3_64.dll"
blpapi 32-bit will be loaded from: "C:\blp\DAPI\blpapi3_32.dll"
System PATH: (* marks locations where blpapi was found)
    "C:\Users\maya.benowitz\AppData\Local\Continuum\anaconda3"
    "C:\Users\maya.benowitz\AppData\Local\Continuum\anaconda3\Library\mingw-w64\bin"
    "C:\Users\maya.benowitz\AppData\Local\Continuum\anaconda3\Library\usr\bin"
    "C:\Users\maya.benowitz\AppData\Local\Continuum\anaconda3\Library\bin"
    "C:\Users\maya.benowitz\AppData\Local\Continuum\anaconda3\Scripts"
    "C:\WINDOWS\system32"
    "C:\WINDOWS"
    "C:\WINDOWS\System32\Wbem"
    "C:\WINDOWS\System32\WindowsPowerShell\v1.0\"
    "C:\Program Files\Docker\Docker\resources\bin"
   